## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os
from datetime import datetime

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 1
data_dir = f"../data/MIMIC/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_12.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_31.pkl",'rb'))
# V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
# W1 = pkl.load(open(data_dir+"X_53.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_23_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_23_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_23_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_23_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"V1":V1,"R":R_temp_dict}

Loading data from data_dir:  ../data/MIMIC/sample1/


In [6]:
print("U1.shape: ",U1.shape)
print("V1.shape: ",V1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (5891, 1321)
V1.shape:  (596, 5891)
R.shape:  (1321, 596)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X3"],\
    "e2":["X1","X2"],\
    "e3":["X2","X3"]
}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":U1},\
    "X2":{"1":data_dict['R'][1]["Rtrain"]},\
    "X3":V1}

In [9]:
X_meta = {
    "X1":["e1","e2"],\
    "X2":["e2","e3"],\
    "X3":["e3","e1"]}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[0, 65, 1], [0, 3, 0]]
Rtest_triplets2 = [[0, 64, 1], [0, 66, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
start_time = datetime.now()

In [19]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X3']
X_id:  X1
X[X_id].shape:  (5891, 1321)
X_id:  X3
X[X_id].shape:  (596, 5891)
C_dict[e].shape:  torch.Size([5891, 1917])
---
e_id:  e2
X_id_list:  ['X1', 'X2']
X_id:  X1
X[X_id].shape:  (5891, 1321)
X_id:  X2
X[X_id].shape:  (1321, 596)
C_dict[e].shape:  torch.Size(

In [20]:
end_time = datetime.now()
runtime = end_time - start_time
runtime_seconds = runtime.total_seconds()
print(f"Total runtime = {runtime_seconds}")

Total runtime = 18.372043


#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [21]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2', 'e3'])

In [22]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3'])

In [23]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [1.549375832080841,
   1.3309117555618286,
   1.785143107175827,
   1.34765625,
   0.6081496179103851,
   1.859401673078537]},
 'loss_all_folds_avg_tuple': [1.549375832080841,
  1.3309117555618286,
  1.785143107175827,
  1.34765625,
  0.6081496179103851,
  1.859401673078537],
 'loss_all_folds_avg_sum': 8.480638235807419,
 'val_metric': 'auc',
 'val_perf_all_folds': {'1': {'X1': 0.0, 'X2': 0.0}},
 'val_perf_all_folds_avg': {'X1': 0.0, 'X2': 0.0},
 'val_perf_all_folds_total': {'1': 0.0},
 'val_perf_all_folds_total_avg': 0.0,
 'E': 3,
 'M': 3}

In [24]:
patient_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
disease_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()
drug_emb = list(dcmf_model.out_dict_U['1'].values())[2].detach().numpy()

In [25]:
import pandas as pd

In [26]:
patient_emb_df = pd.DataFrame(patient_emb)
patient_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.080617,-0.007324,-0.061221,0.020802,-0.119057,-0.128825,-0.099612,0.119814,0.029561,0.014299,...,0.064972,0.147992,-0.097677,0.068118,0.054311,-0.028758,0.104350,0.092996,0.032126,-0.058838
1,-0.079881,-0.001303,-0.060203,0.035042,-0.126175,-0.145234,-0.102229,0.127987,0.030304,0.012941,...,0.076904,0.155431,-0.103693,0.073035,0.050854,-0.040316,0.108829,0.100463,0.034382,-0.056304
2,-0.080688,-0.006301,-0.054046,0.025084,-0.128866,-0.129447,-0.099185,0.120100,0.023127,0.010902,...,0.063975,0.150255,-0.098748,0.067127,0.054190,-0.045451,0.113346,0.089912,0.029862,-0.059678
3,-0.081657,-0.003080,-0.063615,0.029451,-0.121777,-0.140938,-0.099961,0.125315,0.031857,0.011292,...,0.073027,0.155612,-0.099941,0.071054,0.055060,-0.034978,0.107957,0.097141,0.035181,-0.057539
4,-0.080860,-0.003184,-0.063000,0.027543,-0.121962,-0.138548,-0.103258,0.129661,0.028542,0.009106,...,0.072870,0.155571,-0.100250,0.073349,0.055792,-0.035376,0.106730,0.093996,0.030121,-0.054863


In [27]:
patient_emb_df.shape

(5891, 100)

In [28]:
disease_emb_df = pd.DataFrame(disease_emb)
disease_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.126033,0.011271,0.145836,-0.117838,-0.012363,-0.029724,-0.065245,-0.028626,0.039311,-0.189268,...,0.095494,-0.115222,0.137261,0.235593,-0.004376,-0.085078,0.046634,0.013294,-0.189504,0.110355
1,0.076614,-0.010551,0.048096,-0.084747,-0.018700,-0.007771,-0.011970,0.005997,0.044008,-0.098622,...,0.033831,-0.019060,0.099031,0.119764,-0.010995,-0.035944,-0.024371,-0.049600,-0.086027,0.080131
2,0.075541,-0.010946,0.046054,-0.084039,-0.018827,-0.007130,-0.010874,0.006649,0.044132,-0.096421,...,0.032407,-0.016887,0.098191,0.117306,-0.011307,-0.034851,-0.026165,-0.050871,-0.083720,0.079539
3,0.078821,-0.009552,0.052274,-0.086184,-0.018360,-0.008528,-0.014263,0.004240,0.043880,-0.102545,...,0.036631,-0.023303,0.100551,0.124736,-0.010736,-0.037947,-0.021338,-0.046844,-0.090462,0.081312
4,0.092494,-0.003498,0.079570,-0.095489,-0.016174,-0.014796,-0.029166,-0.004974,0.042276,-0.128273,...,0.053849,-0.050322,0.111259,0.157155,-0.008972,-0.051601,-0.001567,-0.029568,-0.119439,0.089747


In [29]:
disease_emb_df.shape

(1321, 100)

In [30]:
drug_emb_df = pd.DataFrame(drug_emb)
drug_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.005670,-0.010648,0.106004,-0.027230,0.079721,0.031052,-0.006859,-0.014729,-0.004371,-0.076216,...,0.080476,-0.143097,0.044231,0.027407,-0.059882,0.021038,0.038276,-0.012123,-0.120387,0.071862
1,-0.008331,-0.014050,0.106581,-0.021412,0.053640,0.011315,-0.021224,-0.003838,0.000564,-0.080029,...,0.091799,-0.118526,0.038085,0.044035,-0.061601,0.022991,0.047120,-0.000500,-0.115073,0.063627
2,-0.008881,-0.014681,0.106580,-0.020426,0.049610,0.008303,-0.023399,-0.002301,0.001382,-0.080676,...,0.093290,-0.114749,0.037065,0.046564,-0.061897,0.023414,0.048423,0.001363,-0.114082,0.062338
3,-0.005141,-0.010020,0.105931,-0.028286,0.084268,0.034494,-0.004337,-0.016679,-0.005266,-0.075531,...,0.078515,-0.147450,0.045317,0.024469,-0.059623,0.020736,0.036716,-0.014173,-0.121337,0.073290
4,-0.006046,-0.010636,0.104777,-0.027750,0.083454,0.033147,-0.004903,-0.016216,-0.004801,-0.074452,...,0.078116,-0.144942,0.044631,0.025230,-0.059367,0.020837,0.036802,-0.013812,-0.120139,0.072281


In [31]:
drug_emb_df.shape

(596, 100)

In [32]:
emb_df = pd.concat([patient_emb_df, disease_emb_df, drug_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(7808, 100)

In [33]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.080617,-0.007324,-0.061221,0.020802,-0.119057,-0.128825,-0.099612,0.119814,0.029561,0.014299,...,0.064972,0.147992,-0.097677,0.068118,0.054311,-0.028758,0.104350,0.092996,0.032126,-0.058838
1,-0.079881,-0.001303,-0.060203,0.035042,-0.126175,-0.145234,-0.102229,0.127987,0.030304,0.012941,...,0.076904,0.155431,-0.103693,0.073035,0.050854,-0.040316,0.108829,0.100463,0.034382,-0.056304
2,-0.080688,-0.006301,-0.054046,0.025084,-0.128866,-0.129447,-0.099185,0.120100,0.023127,0.010902,...,0.063975,0.150255,-0.098748,0.067127,0.054190,-0.045451,0.113346,0.089912,0.029862,-0.059678
3,-0.081657,-0.003080,-0.063615,0.029451,-0.121777,-0.140938,-0.099961,0.125315,0.031857,0.011292,...,0.073027,0.155612,-0.099941,0.071054,0.055060,-0.034978,0.107957,0.097141,0.035181,-0.057539
4,-0.080860,-0.003184,-0.063000,0.027543,-0.121962,-0.138548,-0.103258,0.129661,0.028542,0.009106,...,0.072870,0.155571,-0.100250,0.073349,0.055792,-0.035376,0.106730,0.093996,0.030121,-0.054863


In [34]:
with open(f"emb_MIMIC_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    